In [1]:
!nvidia-smi

Wed May 26 07:52:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive


Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [3]:
%cd ./drive/MyDrive/TB2021/
!ls

/content/drive/MyDrive/TB2021
DataAugmentation.ipynb	models				splited_clean.zip
label_map.csv		Ranger-Deep-Learning-Optimizer	splited_data.rar
label_map.gsheet	splited				splited.zip
__MACOSX		splited_clean_data		test_kaggle


In [ ]:
%cd splited_clean_data/
!ls

/content/drive/MyDrive/TB2021/splited_clean_data
test  train  val


In [ ]:
!gdown --id '1f89r4D09QorqOzdRiylcFps6S1RLnSVn' --output splited_clean.zip 

Downloading...
From: https://drive.google.com/uc?id=1f89r4D09QorqOzdRiylcFps6S1RLnSVn
To: /content/drive/MyDrive/TB2021/splited_clean.zip
146MB [00:00, 158MB/s]


In [ ]:
!unzip -O big5 splited_clean.zip


Streaming output truncated to the last 5000 lines.
  inflating: splited_clean_data/val/48967_isnull.jpg  
  inflating: __MACOSX/splited_clean_data/val/._48967_isnull.jpg  
  inflating: splited_clean_data/val/2943_俊.jpg  
  inflating: __MACOSX/splited_clean_data/val/._2943_俊.jpg  
  inflating: splited_clean_data/val/16167_治.jpg  
  inflating: __MACOSX/splited_clean_data/val/._16167_治.jpg  
  inflating: splited_clean_data/val/50622_際.jpg  
  inflating: __MACOSX/splited_clean_data/val/._50622_際.jpg  
  inflating: splited_clean_data/val/15645_佑.jpg  
  inflating: __MACOSX/splited_clean_data/val/._15645_佑.jpg  
  inflating: splited_clean_data/val/12743_歐.jpg  
  inflating: __MACOSX/splited_clean_data/val/._12743_歐.jpg  
  inflating: splited_clean_data/val/16558_青.jpg  
  inflating: __MACOSX/splited_clean_data/val/._16558_青.jpg  
  inflating: splited_clean_data/val/10983_顏.jpg  
  inflating: __MACOSX/splited_clean_data/val/._10983_顏.jpg  
  inflating: splited_clean_data/val/22513_霞.jpg  
  i

In [ ]:
%cd splited_clean_data

/content/drive/My Drive/TB2021/splited_clean_data


## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [ ]:
####流程
# 1.載洛翔的 clean data
# 2.unzip big5(避免編碼錯誤)
# 3.cd到那個資料夾
# 4.error_list是錯誤的data
# 5.get_label_dict是創建label_map
# 6.read_directory是創建子資料夾
# 結束後會有train_label,test_label,val_label三個 裡面會各有000~800的子資料夾 可以給DatasetFolder吃

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [ ]:
error_list = ['10020',
 '10028',
 '10062',
 '10141',
 '10151',
 '10171',
 '10225',
 '10226',
 '10321',
 '10470',
 '10495',
 '10536',
 '10541',
 '10600',
 '10793',
 '10817',
 '10831',
 '10845',
 '10890',
 '11013',
 '11132',
 '11143',
 '11176',
 '11200',
 '11207',
 '11327',
 '11358',
 '11476',
 '11495',
 '11515',
 '11534',
 '11608',
 '11620',
 '11674',
 '11685',
 '11797',
 '11819',
 '11842',
 '11877',
 '11916',
 '11927',
 '11973',
 '12034',
 '12100',
 '12115',
 '12120',
 '12185',
 '12264',
 '12387',
 '12397',
 '12505',
 '12596',
 '12668',
 '12705',
 '12765',
 '12795',
 '12801',
 '12840',
 '12860',
 '12941',
 '12996',
 '13012',
 '13013',
 '13113',
 '13128',
 '13132',
 '13147',
 '13165',
 '13205',
 '13262',
 '13264',
 '13321',
 '13329',
 '13349',
 '13378',
 '13388',
 '13558',
 '13586',
 '13620',
 '13638',
 '13695',
 '13696',
 '13760',
 '13781',
 '13783',
 '13824',
 '13842',
 '13846',
 '13875',
 '13898',
 '13904',
 '13957',
 '14032',
 '14071',
 '14084',
 '14100',
 '14179',
 '14194',
 '14213',
 '14297',
 '14315',
 '14334',
 '14399',
 '14406',
 '14417',
 '14441',
 '14482',
 '14540',
 '14582',
 '14617',
 '14696',
 '14760',
 '14869',
 '14922',
 '14936',
 '14940',
 '14946',
 '15030',
 '15034',
 '15135',
 '15202',
 '15203',
 '15210',
 '15228',
 '15278',
 '15314',
 '15328',
 '15345',
 '15357',
 '15586',
 '15667',
 '15670',
 '15737',
 '15740',
 '15756',
 '15818',
 '15853',
 '15863',
 '15988',
 '16040',
 '16041',
 '16137',
 '16145',
 '16164',
 '16169',
 '16205',
 '16216',
 '16244',
 '16255',
 '16260',
 '16330',
 '16359',
 '16381',
 '16382',
 '16402',
 '16414',
 '16424',
 '16468',
 '16471',
 '16500',
 '16512',
 '16589',
 '16600',
 '16603',
 '16630',
 '16660',
 '16742',
 '16827',
 '16839',
 '16842',
 '16853',
 '16896',
 '16975',
 '17038',
 '17076',
 '17107',
 '17176',
 '17178',
 '17205',
 '17235',
 '17246',
 '17394',
 '17420',
 '17525',
 '17532',
 '17533',
 '17643',
 '17654',
 '17717',
 '17820',
 '17822',
 '17835',
 '17857',
 '17876',
 '17910',
 '17940',
 '17942',
 '18003',
 '18116',
 '18118',
 '18125',
 '18128',
 '18134',
 '18220',
 '18236',
 '18276',
 '18294',
 '18315',
 '18319',
 '18321',
 '18353',
 '18363',
 '18404',
 '18426',
 '18454',
 '18502',
 '18550',
 '18559',
 '18589',
 '18659',
 '18663',
 '18732',
 '18745',
 '18751',
 '18757',
 '18776',
 '18782',
 '18805',
 '18865',
 '18894',
 '18981',
 '19044',
 '19063',
 '19093',
 '19108',
 '19250',
 '19309',
 '19310',
 '19373',
 '19411',
 '19439',
 '19589',
 '19625',
 '19668',
 '19693',
 '19766',
 '19794',
 '19822',
 '19836',
 '19910',
 '19918',
 '19922',
 '19929',
 '19944',
 '19946',
 '19961',
 '19991',
 '27065',
 '27177',
 '27178',
 '27249',
 '27261',
 '27304',
 '27352',
 '27358',
 '27460',
 '27570',
 '27635',
 '27652',
 '27670',
 '27688',
 '27697',
 '27712',
 '27737',
 '27799',
 '27808',
 '27824',
 '27983',
 '28009',
 '28014',
 '28024',
 '28043',
 '28070',
 '28097',
 '28100',
 '28201',
 '28277',
 '28322',
 '28327',
 '28382',
 '28436',
 '28542',
 '28544',
 '28579',
 '28663',
 '28674',
 '28731',
 '28799',
 '28816',
 '28843',
 '28859',
 '28899',
 '28907',
 '28910',
 '29041',
 '29063',
 '29180',
 '29266',
 '29356',
 '29451',
 '29488',
 '29518',
 '29528',
 '29584',
 '29710',
 '29736',
 '29771',
 '29869',
 '29927',
 '29937',
 '29948',
 '30978',
 '36917']

In [ ]:
def get_label_dict(directory_name):
    total = 0
    label_dict = {}
    one_hot_count = 0
    error_flag = False
    ## 先統計哪些label 建dict
    for filename in os.listdir(r"./"+directory_name):
        for error in error_list:
            if error in filename:
                error_flag = True
        if not error_flag:
            label = filename[-5]
            if label not in label_dict:
                label_dict[label] = one_hot_count
                one_hot_count += 1
        error_flag = False
    print("dict done!")
    print(len(label_dict))
    print(one_hot_count)
    print(label_dict)
    return label_dict

In [ ]:
# this function is for read image,the input is directory name
def read_directory(directory_name,label_dict):
    ## 重建方便DatasetFolder
    label_count = [0 for i in range(len(label_dict))]
    os.makedirs(directory_name + "_label",exist_ok=True)
    for word in label_dict:
        os.makedirs(directory_name + "_label/"+ str(label_dict[word]).zfill(3),exist_ok=True)
    check = 0
    for filename in os.listdir(r"./"+directory_name):
        label = filename[-5]
        if label in label_dict: ##先不管沒出現的
            num_label = label_dict[label]
            img = cv2.imread(directory_name + "/" + filename)
            cv2.imwrite(directory_name + "_label/"+ str(num_label).zfill(3) + "/" + str(num_label) + "_" +str(label_count[num_label]) +".jpg",img)
            label_count[num_label] += 1
            check += 1
            if check % 500 == 0:
                print(check) 

In [ ]:
!ls

test  train  val


In [ ]:
label_dict = get_label_dict("train")

dict done!
801
801
{'巨': 0, '薇': 1, '榕': 2, '室': 3, '圓': 4, '觀': 5, '橋': 6, '特': 7, '梅': 8, '務': 9, '琪': 10, 'l': 11, '穎': 12, '心': 13, '問': 14, '仕': 15, '田': 16, '涵': 17, '逸': 18, '術': 19, '兒': 20, '地': 21, '松': 22, '鳳': 23, '優': 24, '尚': 25, '曜': 26, '欣': 27, '區': 28, '東': 29, '石': 30, '之': 31, '璟': 32, '北': 33, '好': 34, '健': 35, '專': 36, '宏': 37, '英': 38, '鏡': 39, '豪': 40, '沈': 41, '卓': 42, '銓': 43, '暘': 44, '建': 45, '餐': 46, '喬': 47, '儒': 48, '曉': 49, '語': 50, '呂': 51, '人': 52, '蓁': 53, '周': 54, '珠': 55, '黃': 56, '券': 57, '盟': 58, '發': 59, '簡': 60, '呈': 61, '范': 62, '瑜': 63, '薛': 64, '藝': 65, '祥': 66, '傳': 67, '民': 68, '震': 69, '工': 70, '豐': 71, '閔': 72, '芷': 73, '位': 74, '施': 75, '翠': 76, '鈞': 77, '書': 78, '經': 79, '眾': 80, '數': 81, '權': 82, '晶': 83, '朝': 84, '強': 85, '名': 86, '保': 87, '社': 88, '男': 89, '維': 90, '真': 91, '生': 92, '妤': 93, '忠': 94, '加': 95, '金': 96, '本': 97, '康': 98, '房': 99, '丁': 100, '策': 101, '義': 102, '秀': 103, '紘': 104, '貴': 105, '水': 106, '可': 107, '院': 108, 

In [ ]:
read_directory("train",label_dict)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500


In [ ]:
read_directory("test",label_dict)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500


In [ ]:
read_directory("val",label_dict)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500


In [ ]:
!ls

test  test_label  train  train_label  val  val_label


In [ ]:
%cd ..

/content/drive/My Drive/TB2021


In [4]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import os
import cv2
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# This is for the progress bar.
from tqdm.auto import tqdm

In [ ]:
result_file = "label_map.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Word\n")
	  for i, word in enumerate(label_dict):
		    f.write(f"{str(i)},{word}\n")

In [35]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((96, 96)),
    # transforms.RandomCrop((84,84)),
    # transforms.Resize((96, 96)),
    # transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    # transforms.RandomRotation((-15,15)),

    # You may add some transforms here.
    # transforms.RandomHorizontalFlip(0.5),
    # transforms.RandomCrop((112,112)),
    # transforms.Resize((128, 128)),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])
# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
])


In [36]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully. 
batch_size = 128

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("splited_clean_data/train_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("splited_clean_data/val_label", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("splited_clean_data/test_label", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.



In [27]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.AvgPool2d(kernel_size, stride, padding)
        # torch.nn.AdaptiveAvgPool2d = keras' GlobalAvgPooling

        # input image size: [3, 96, 96]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),

            nn.Conv2d(64, 96, 3, 1, 1, bias = False),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 64, 3, 1, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 96, 3, 1, 1, bias = False),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),

            nn.Conv2d(96, 128, 3, 1, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 96, 3, 1, 1, bias = False),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 128, 3, 1, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 192, 3, 1, 1, bias = False),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.Conv2d(192, 256, 3, 1, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),

            nn.Conv2d(256, 448, 3, 1, 1, bias = False),
            nn.BatchNorm2d(448),
            nn.ReLU(),
            nn.Conv2d(448, 256, 3, 1, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 448, 3, 1, 1, bias = False),
            nn.BatchNorm2d(448),
            nn.ReLU(),
            nn.AvgPool2d(3, 2, 0),


            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.5)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(448, 801)
        )
    def forward(self, x):
        # input (x): [batch_size, 3, 96, 96]
        # output: [batch_size, 801]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)
        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)
        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [8]:
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 15.5MB/s 
     |████████████████████████████████| 901kB 54.3MB/s 
     |████████████████████████████████| 3.3MB 49.9MB/s 


In [10]:
# !git clone https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer
%cd Ranger-Deep-Learning-Optimizer
!pip install -e . 

/content/drive/My Drive/TB2021/Ranger-Deep-Learning-Optimizer
Obtaining file:///content/drive/My%20Drive/TB2021/Ranger-Deep-Learning-Optimizer
  Running setup.py develop for ranger


In [11]:
%cd ..

/content/drive/My Drive/TB2021


In [1]:
from transformers import get_linear_schedule_with_warmup
# from ranger import Ranger  # this is from ranger.py
# from ranger import RangerVA  # this is from ranger913A.py
# from ranger import RangerQH  # this is from rangerqh.py

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()
# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3,betas = (0.9,0.98),eps = 1e-6)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = Ranger(model.parameters(),lr=2e-3)
# label_count = [0,0,0,0,0,0,0,0,0,0,0]
# The number of training epochs.
n_epochs = 100
model_path = './models/CNN_SGD_1e2_128.ckpt'
total_step = len(train_loader) * n_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,1,total_step)

# model.load_state_dict(torch.load(model_path)
best_acc = 0.0
step = 1

for epoch in range(n_epochs):
    
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []
    # if epoch >= AdamW_epochs:
    #     optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    #     scheduler = get_linear_schedule_with_warmup(optimizer,1,SGD_step)
    # Iterate the training set by batches.
    for batch_idx, batch in enumerate(tqdm(train_loader)):
        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits.to(device), labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.A
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()
        scheduler.step()
            # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    if valid_acc > best_acc:
        torch.save(model.state_dict(),model_path)
        print('save model with acc:',valid_acc)
        best_acc = valid_acc

NameError: ignored

In [15]:
model = Classifier().to(device)
model_path = './models/CNN_1e2_SGD_256.ckpt'
model.load_state_dict(torch.load(model_path))
model

Classifier(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): AvgPool2d(kernel_size=3, stride=2, padding=0)
    (7): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(96, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (14): BatchNorm2d(96, eps=1e-05, momentum=0

## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [19]:
# this function is for read image,the input is directory name
def new_read_directory(directory_name,label_dict):
    ## 重建方便DatasetFolder
    label_count = [0 for i in range(len(label_dict))]
    os.makedirs(directory_name + "_kaggle",exist_ok=True)
    for word in label_dict:
        os.makedirs(directory_name + "_kaggle/"+ str(label_dict[word]).zfill(3),exist_ok=True)
    check = 0
    for filename in os.listdir(r"./"+directory_name):
        label = filename[-5]
        name = filename.split('_')[0]
        
        if label in label_dict: ##先不管沒出現的
            num_label = label_dict[label]
            img = cv2.imread(directory_name + "/" + filename)
            cv2.imwrite(directory_name + "_kaggle/"+ str(num_label).zfill(3) + "/" + name +".jpg",img)
            label_count[num_label] += 1
            check += 1
            if check % 500 == 0:
                print(check) 

In [29]:
import pandas as pd
def get_label_map():
    label2word = {}
    map = pd.read_csv('./label_map.csv')
    for idx,word in enumerate(map['Word']):
        label2word[word] = idx
    return label2word


label_dict = get_label_map()

In [33]:
new_read_directory("./splited_clean_data/test",label_dict)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-654e34a766d1>", line 1, in <module>
    new_read_directory("./splited_clean_data/test",label_dict)
  File "<ipython-input-19-9e6e9168fecc>", line 9, in new_read_directory
    for filename in os.listdir(r"./"+directory_name):
FileNotFoundError: [Errno 2] No such file or directory: '././splited_clean_data/test'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.

FileNotFoundError: ignored

In [16]:
ori_test_set = DatasetFolder("splited_clean_data/test", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.

ori_test_loader = DataLoader(ori_test_set, batch_size=batch_size, shuffle=False)

RuntimeError: ignored

In [13]:
model.eval()
test_loss = []
test_accs = []
for batch in tqdm(test_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch

    # We don't need gradient in validation.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
      logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
    loss = criterion(logits, labels.to(device))

    # Compute the accuracy for current batch.
    acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
    test_loss.append(loss.item())
    test_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
test_loss = sum(test_loss) / len(test_loss)
test_acc = sum(test_accs) / len(test_accs)

print('test_loss:',test_loss)
print('test_acc:',test_acc.item())


test_loss: 0.5591199163485456
test_acc: 0.8995434045791626


In [ ]:
from PIL import Image
import cv2
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Classifier().to(device)
model_path = 'CNN_1e2_SGD_256.ckpt'
model.load_state_dict(torch.load(model_path))

def get_label_map():
    label2word = {}
    map = pd.read_csv('./label_map.csv')
    for idx,word in enumerate(map['Word']):
        label2word[idx] = word
    return label2word


label_map = get_label_map()

### img : 是主辦給的cv2檔 
### label_map : 把pred轉成word
def model_predict(model, label_map, img):
    ## Convert cv2 to PIL
    PIL_img = Image.fromarray(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    with torch.no_grad():
        logits = model(PIL_img.to(device))
    pred = torch.argmax(logits,dim=-1)
    return label_map[pred]

result = model_predict(model,label_map,img)

In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
def base64_to_binary_for_cv2(image_64_encoded):
    """ Convert base64 to numpy.ndarray for cv2.

    @param:
        image_64_encode(str): image that encoded in base64 string format.
    @returns:
        image(numpy.ndarray): an image.
    """
    img_base64_binary = image_64_encoded.encode("utf-8")
    img_binary = base64.b64decode(img_base64_binary)
    image = cv2.imdecode(np.frombuffer(img_binary, np.uint8), cv2.IMREAD_COLOR)
    return image

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")